In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import numpy as np
import pandas as pd

In [2]:
io_coeff_table = pd.read_csv("2017_io_coeff_matrix.csv")
io_coeff_table.columns = io_coeff_table.columns.map(lambda a: "c" + a[1:])
io_coeff_table.index = io_coeff_table.columns

In [3]:
final_demand = pd.read_csv("2017_final_demand.csv",header=None)
final_demand.index = io_coeff_table.index

In [4]:
io_coeff_mat = np.matrix(io_coeff_table)

In [5]:
io_coeff_mat.shape

(35, 35)

In [6]:
interact_args = dict.fromkeys(io_coeff_table.index,(0.0,2.0,0.01))

In [7]:
def total_primary_inputs_percentage(c_mat, fin_dem, **percentages):
    p_list = np.array(list(percentages.values())).reshape(-1,1)
    
    total_inputs_100 = np.linalg.solve(np.identity(c_mat.shape[0]) - c_mat, fin_dem)
    
    total_inputs_scaled_by_p = np.linalg.solve(np.identity(c_mat.shape[0]) - c_mat, np.multiply(p_list,fin_dem))
    
    gva_rate = np.transpose(1.0 - c_mat.sum(axis=0))
    
    gva_100 = np.multiply(gva_rate,total_inputs_100)
    gva_scaled_by_p = np.multiply(gva_rate,total_inputs_scaled_by_p)
    
    print("Original Total Gross Value Added ($ million)")
    print(np.sum(gva_100))
    
    print("Shocked Total Gross Value Added ($ million)")
    print(np.sum(gva_scaled_by_p))
    display(pd.DataFrame(np.concatenate((gva_100,gva_scaled_by_p),axis=1),index=io_coeff_table.index,columns=['original_gva','shocked_gva']))

In [8]:
# 2 decimal point precision and suppress scientific notation
np.set_printoptions(precision=2, suppress=True)

In [9]:
interact(total_primary_inputs_percentage, c_mat = fixed(io_coeff_mat), fin_dem = fixed(np.array(final_demand)), **interact_args)

interactive(children=(FloatSlider(value=1.0, description='c1', max=2.0, step=0.01), FloatSlider(value=1.0, des…

<function __main__.total_primary_inputs_percentage(c_mat, fin_dem, **percentages)>

In [10]:
w = interactive(total_primary_inputs_percentage, c_mat = fixed(io_coeff_mat), fin_dem = fixed(np.array(final_demand)), **interact_args)

In [11]:
for i in range(0,35):
    w.children[i].orientation = 'vertical'

In [12]:
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')

box_1 = widgets.Box(w.children[0:12], layout = box_layout)
box_2 = widgets.Box(w.children[12:24], layout = box_layout)
box_3 = widgets.Box(w.children[24:35], layout = box_layout)

In [13]:
ui = widgets.VBox([box_1,box_2,box_3])

In [14]:
interactive_args_2 = dict(zip(list(io_coeff_table.index),w.children[:-1]))
interactive_args_2['c_mat'] = fixed(io_coeff_mat)
interactive_args_2['fin_dem'] = fixed(np.array(final_demand))

In [15]:
out = widgets.interactive_output(total_primary_inputs_percentage, interactive_args_2)

In [16]:
display(ui, out)

Output()